In [1]:
#encoding:utf-8
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import pickle
import numpy as np
import os
from sklearn import metrics
import importlib
import adv_module
importlib.reload(adv_module)
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
import codecs
import split
from functools import reduce

all_f1= []
all_pre = []
all_recall = []
all_accu = []
for i in range(10):
    split.split('../../Data/emnlp_train.txt','../../Data/emnlp_test.txt')
    split.split('../../Data/ntcir_train.txt',language='E')
    split.process()
    
    
    word2vec_C = pickle.load(open('../../Data/emnlp_embedding.pkl', 'rb'))
    word2vec_E = pickle.load(open('../../Data/ntcir_embedding.pkl', 'rb'))
    
    F = adv_module.featureExtractor(word2vec_E,word2vec_C, embedding_trainable=False)
    D = adv_module.discriminator()
    C = adv_module.classifier()
    F.cuda()
    D.cuda()
    C.cuda()
    dataC = pickle.load(open('../../Data/emnlp_train.pkl', 'rb'))
    testC = pickle.load(open('../../Data/emnlp_test.pkl', 'rb'))
    dataE = pickle.load(open('../../Data/ntcir_train.pkl', 'rb'))
    testE = pickle.load(open('../../Data/ntcir_test.pkl', 'rb'))
    ###########加载扩增的英文以及中文数据#######
    expandDataE = pickle.load(open('../../Data/ntcir_expand.pkl','rb'))
    expandDataC = pickle.load(open('../../Data/emnlp_expand.pkl','rb'))
    #################################
#     F_param = list(F.parameters())
#     D_param = list(D.parameters())
#     C_param = list(C.parameters())
#     F_params = filter(lambda x: x.requires_grad, F_param)
#     D_params = filter(lambda x: x.requires_grad, D_param)
#     C_params = filter(lambda x: x.requires_grad, C_param)
  
#     D_opt = optim.Adam(D_params, lr=0.001) 
#     C_opt = optim.Adam(C_params,lr=0.001)
#     F_opt = optim.Adam(F_params,lr=0.001)
    
#     #############扩增后模型###################
#     #################################
    F_param = list(F.parameters())
    D_param = list(D.parameters())
    C_param = list(C.parameters())
    params = filter(lambda x: x.requires_grad, F_param+D_param+C_param)
    train_opt = optim.Adam(params, lr=0.002)
    
    ####################################
    crossentropy = nn.CrossEntropyLoss()
    epochs = 2
    max_f1 = 0    
    num_batches = len(dataE)
    for epoch in range(epochs):
        for i, batch_data in enumerate(dataE):
            F.train(mode=True)
            D.train(mode=True)
            C.train(mode=True)
            
            train_opt.zero_grad()
#             C_opt.zero_grad()
#             F_opt.zero_grad()
#             D_opt.zero_grad()
        
            textE, emoE, positionE, labelE, _, _ = zip(*batch_data)
            textC, emoC, positionC, labelC, _, _  = zip(*dataC[i])
            ##########################
            textCE, emoCE, positionCE, labelCE, _, _  = zip(*expandDataC[i])
            textEC, emoEC, positionEC, labelEC, _, _  = zip(*expandDataE[i])

            #############################
            D_realE = np.array([[0,1]]*len(textE))
            D_realC = np.array([[1,0]]*len(textC))
#             D_fakeE = np.array([[1,0]]*len(textE))
#             D_fakeC = np.array([[0,1]]*len(textC))
        
            D_realE = np.array([[1,0,0,0]]*len(textE))
            D_realC = np.array([[0,1,0,0]]*len(textC))
            D_realEC = np.array([[0,0,1,0]]*len(textEC))
            D_realCE = np.array([[0,0,0,1]]*len(textCE))
            
            _D_realEC = Variable(torch.LongTensor(np.argmax(D_realEC,1)).cuda())
            _D_realCE = Variable(torch.LongTensor(np.argmax(D_realCE,1)).cuda())
            _D_realE = Variable(torch.LongTensor(np.argmax(D_realE,1)).cuda())
            _D_realC = Variable(torch.LongTensor(np.argmax(D_realC,1)).cuda())
#             _D_fakeE = Variable(torch.LongTensor(np.argmax(D_fakeE,1)).cuda())
#             _D_fakeC = Variable(torch.LongTensor(np.argmax(D_fakeC,1)).cuda())
           
            
            textE, emoE, positionE,labelE = np.array(textE), np.array(list(emoE)), np.array(positionE), np.array(labelE)
            textC, emoC, positionC,labelC = np.array(textC), np.array(list(emoC)), np.array(positionC), np.array(labelC)
        
            
            ####################################
            textEC, emoEC, positionEC,labelEC = np.array(textEC), np.array(list(emoEC)), np.array(positionEC), np.array(labelEC)
            textCE, emoCE, positionCE,labelCE = np.array(textCE), np.array(list(emoCE)), np.array(positionCE), np.array(labelCE)
            #######################################
            batch_size = textE.shape[0]+textC.shape[0]
          
            _textE = Variable(torch.LongTensor(textE).cuda())
            _emoE = Variable(torch.LongTensor(emoE).cuda())
            _posE = Variable(torch.FloatTensor(positionE).cuda())
            _labelE = Variable(torch.LongTensor(np.argmax(labelE, 1)).cuda())
            _textC = Variable(torch.LongTensor(textC).cuda())
            _emoC = Variable(torch.LongTensor(emoC).cuda())
            _posC = Variable(torch.FloatTensor(positionC).cuda())
            _labelC = Variable(torch.LongTensor(np.argmax(labelC, 1)).cuda())
            ########################################
            _textEC = Variable(torch.LongTensor(textEC).cuda())
            _emoEC = Variable(torch.LongTensor(emoEC).cuda())
            _posEC = Variable(torch.FloatTensor(positionEC).cuda())
            _labelEC = Variable(torch.LongTensor(np.argmax(labelEC, 1)).cuda())
            _textCE = Variable(torch.LongTensor(textCE).cuda())
            _emoCE = Variable(torch.LongTensor(emoCE).cuda())
            _posCE = Variable(torch.FloatTensor(positionCE).cuda())
            _labelCE = Variable(torch.LongTensor(np.argmax(labelCE, 1)).cuda())
     
            # extract  features 
            F_textE,F_textC,F_textEC,F_textCE, F_emoE,F_emoC,F_emoEC,F_emoCE = F(textE=_textE,textC=_textC,emoE=_emoE,emoC=_emoC,textEC = _textEC,emoEC = _emoEC,textCE = _textCE,emoCE=_emoCE,train='True')
            #train classifier
            C_hidden = C.init_hidden(batch_size)
            C_logits, C_probs = C(textE=F_textE, emoE=F_emoE,posE= _posE,init_state=C_hidden,textC=F_textC, emoC=F_emoC,posC= _posC,train='True')
            C_loss = crossentropy(C_logits, torch.cat([_labelE,_labelC],0))
            
            p = float(i+1 + (epoch+1)*num_batches/epochs/num_batches)
            alpha = 2. / (1. + np.exp(-10 * p)) - 1
            
            D_logits,D_probs = D(F_textE,F_emoE,F_textC, F_emoC,F_textEC,F_emoEC,F_textCE,F_emoCE,alpha)  
            D_loss = crossentropy(D_logits,torch.cat([_D_realE,_D_realC,_D_realEC,_D_realCE],0))
            #train feature extractor
            (C_loss+D_loss).backward()
            train_opt.step()
           
            
      
            ############################################
            
#             # extract  features 
#             F_textE,F_textC, F_emoE,F_emoC= F(textE=_textE,emoE=_emoE,textC=_textC,emoC=_emoC)
#             #train classifier
#             C_hidden = C.init_hidden(batch_size)
#             #print('32222',F_textE.size(),'22',F_textC.size(),'55', F_emoE.size(),'77',F_emoC.size())
#             C_logits, C_probs = C(textE=F_textE, emoE=F_emoE,posE= _posE,init_state=C_hidden,train='True')
    
#             C_loss = crossentropy(C_logits, _labelC)
#             C_loss.backward(retain_variables=True)
#             C_opt.step()
#             #train discriminator
#             D_logits,D_probs = D(F_textE,F_emoE,F_textC, F_emoC)
#             D_loss = crossentropy(D_logits,torch.cat([_D_realE,_D_realC],0))
#             D_loss.backward(retain_variables=True)
#             D_opt.step()
#             #train feature extractor
#             D_loss_fake = crossentropy(D_logits,torch.cat([_D_fakeE,_D_fakeC],0))
#             F_opt.zero_grad()
#             F_loss = D_loss_fake + 0.7*C_loss
#             F_loss.backward()
#             F_opt.step()
            i += 1
            if i%1 == 0:
                preds = []
                pred_probs = []
                test_label = []
                inxs = []
                clause_inxs = []
        
                for j,batch_data in enumerate(testC):
                   
                    F.eval()
                    D.eval()
                    C.eval()  
                    
                    
                    textE, emoE, positionE, labelE, _,_ = zip(*testE[j])
                    textC, emoC, positionC, labelC, inxsC, clause_inxC  = zip(*batch_data)
                    
#                     D_realE = np.array([[0,1]]*len(textE))
#                     D_realC = np.array([[1,0]]*len(textC))
#                     D_fakeE = np.array([[1,0]]*len(textE))
#                     D_fakeC = np.array([[0,1]]*len(textC))
#                     _D_realE = Variable(torch.LongTensor(np.argmax(D_realE,1)).cuda())
#                     _D_realC = Variable(torch.LongTensor(np.argmax(D_realC,1)).cuda())
#                     _D_fakeE = Variable(torch.LongTensor(np.argmax(D_fakeE,1)).cuda())
#                     _D_fakeC = Variable(torch.LongTensor(np.argmax(D_fakeC,1)).cuda())
                
                    textE, emoE, positionE,labelE = np.array(textE), np.array(list(emoE)), np.array(positionE), np.array(labelE)
                    textC, emoC, positionC,labelC = np.array(textC), np.array(list(emoC)), np.array(positionC), np.array(labelC)
                
                    _textE = Variable(torch.LongTensor(textE).cuda())
                    _emoE = Variable(torch.LongTensor(emoE).cuda())
                    _posE = Variable(torch.FloatTensor(positionE).cuda())
                  
                    _textC = Variable(torch.LongTensor(textC).cuda())
                    _emoC = Variable(torch.LongTensor(emoC).cuda())
                    _posC = Variable(torch.FloatTensor(positionC).cuda())
                    
#                     F_textE,F_textC,F_emoE,F_emoC = F(_textE,_emoE,_textC,_emoC)
#                    batch_size = F_textC.size()[0]
    
#                     C_hidden = C.init_hidden(batch_size)
#                     C_logits,C_probs = C(init_state=C_hidden,textC=F_textC,emoC=F_emoC,posC= _posC, train='False')
#                     ###################################
                    F_textC,F_emoC = F(textC=_textC,emoC=_emoC,train='False')
                    batch_size = F_textC.size()[0]
                    C_hidden = C.init_hidden(batch_size)
                    C_logits,C_probs = C(init_state = C_hidden,textC = F_textC,emoC = F_emoC, posC = _posC, train='False')
                    ####################################
                    pred = list(np.argmax(C_probs.data.cpu().numpy(), 1))  
                    preds.extend(pred)
                    test_label.extend(list(np.argmax(labelC, 1)))
                    inxs.extend(inxsC)
                    clause_inxs.extend(clause_inxC)
                    pred_probs.extend(C_probs.data.cpu().numpy()[:, 1])
               
                acc = metrics.accuracy_score(np.array(test_label), np.array(preds))
                pre = metrics.precision_score(np.array(test_label), np.array(preds))#, average='macro'
                rec = metrics.recall_score(np.array(test_label), np.array(preds))
                f1 = metrics.f1_score(np.array(test_label), np.array(preds))
                if f1 > max_f1:
                    max_f1 = f1
                    max_acc = acc
                    max_pre = pre
                    max_rec = rec
                    max_epoch = epoch
                    if max_f1 > 0.30:
                        file_name = 'expand_f1_'+str(f1) + '_pre_'+str(pre)+'_rec_'+str(rec)+'_epoch_'+str(epoch)+'_'+str(0.7)
                        with open('../../Data/Prediction/' + file_name, 'w') as f_w:
                            for inx_i in range(len(inxs)):
                                f_w.write(inxs[inx_i] + ' ' + clause_inxs[inx_i] + ' ' + str(pred_probs[inx_i]) + '\n')
                print ('epoch: %d, accuracy: %f, precision: %f, recall: %f, f1: %f, loss:%f'%(epoch, acc, pre, rec, f1, C_loss.data[0]))
    print ('---max---epoch: %d, accuracy: %f, precision: %f, recall: %f, f1: %f---max---'%(max_epoch, max_acc, max_pre, max_rec, max_f1))
    all_f1.append(max_f1)
    all_pre.append(max_pre)
    all_recall.append(max_rec)
    all_accu.append(max_acc)
print(all_f1)
print(all_pre)
print(all_recall)
print(all_accu)
f1 = reduce(lambda x,y:x+y,all_f1)/10
recall = reduce(lambda x,y:x+y,all_recall)/10
precision = reduce(lambda x,y:x+y,all_pre)/10
accu = reduce(lambda x,y:x+y,all_accu)/10
print(f1,recall,precision,accu)


2105


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.567 seconds.
Prefix dict has been built succesfully.
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


epoch: 0, accuracy: 0.927217, precision: 0.463609, recall: 0.500000, f1: 0.481117, loss:0.737251
epoch: 0, accuracy: 0.927217, precision: 0.463609, recall: 0.500000, f1: 0.481117, loss:0.599077
epoch: 0, accuracy: 0.927217, precision: 0.463609, recall: 0.500000, f1: 0.481117, loss:0.379488
epoch: 0, accuracy: 0.927217, precision: 0.463609, recall: 0.500000, f1: 0.481117, loss:0.245659
epoch: 0, accuracy: 0.927217, precision: 0.463609, recall: 0.500000, f1: 0.481117, loss:0.098071
epoch: 0, accuracy: 0.927217, precision: 0.463609, recall: 0.500000, f1: 0.481117, loss:0.086374
epoch: 0, accuracy: 0.927217, precision: 0.463609, recall: 0.500000, f1: 0.481117, loss:0.116583
epoch: 0, accuracy: 0.927217, precision: 0.463609, recall: 0.500000, f1: 0.481117, loss:0.147706
epoch: 0, accuracy: 0.927217, precision: 0.463609, recall: 0.500000, f1: 0.481117, loss:0.325809
epoch: 0, accuracy: 0.927217, precision: 0.463609, recall: 0.500000, f1: 0.481117, loss:0.000518
epoch: 0, accuracy: 0.927217, 

epoch: 0, accuracy: 0.923832, precision: 0.636967, recall: 0.521748, f1: 0.524662, loss:0.281660
epoch: 0, accuracy: 0.925863, precision: 0.704709, recall: 0.572133, f1: 0.598986, loss:0.437322
epoch: 0, accuracy: 0.921801, precision: 0.701244, recall: 0.659950, f1: 0.677528, loss:0.378562
epoch: 0, accuracy: 0.920447, precision: 0.697889, recall: 0.667792, f1: 0.681218, loss:0.285511
epoch: 0, accuracy: 0.915708, precision: 0.692536, recall: 0.708097, f1: 0.699911, loss:0.298118
epoch: 0, accuracy: 0.830399, precision: 0.606680, recall: 0.743530, f1: 0.628785, loss:0.625610
epoch: 0, accuracy: 0.894381, precision: 0.665708, recall: 0.760887, f1: 0.698037, loss:0.390355
epoch: 0, accuracy: 0.920447, precision: 0.707348, recall: 0.717081, f1: 0.712067, loss:0.311046
epoch: 0, accuracy: 0.927894, precision: 0.726002, recall: 0.583943, f1: 0.615060, loss:0.299086
epoch: 0, accuracy: 0.926879, precision: 0.663886, recall: 0.504104, f1: 0.490104, loss:0.310872
epoch: 0, accuracy: 0.926879, 

epoch: 0, accuracy: 0.921462, precision: 0.707260, recall: 0.698342, f1: 0.702672, loss:0.376648
epoch: 0, accuracy: 0.897089, precision: 0.657502, recall: 0.717344, f1: 0.680591, loss:0.440208
epoch: 0, accuracy: 0.849695, precision: 0.612298, recall: 0.726076, f1: 0.637538, loss:0.283906
epoch: 0, accuracy: 0.596141, precision: 0.541017, recall: 0.649352, f1: 0.466777, loss:0.491712
epoch: 0, accuracy: 0.460393, precision: 0.526380, recall: 0.595437, f1: 0.384714, loss:0.536752
epoch: 0, accuracy: 0.497630, precision: 0.530200, recall: 0.611231, f1: 0.407893, loss:0.325789
epoch: 0, accuracy: 0.588016, precision: 0.540226, recall: 0.647114, f1: 0.462152, loss:0.518749
epoch: 0, accuracy: 0.759986, precision: 0.570086, recall: 0.699131, f1: 0.565975, loss:0.461584
epoch: 0, accuracy: 0.854435, precision: 0.610729, recall: 0.711487, f1: 0.635206, loss:0.344507
epoch: 0, accuracy: 0.887610, precision: 0.637420, recall: 0.697231, f1: 0.659398, loss:0.238335
epoch: 0, accuracy: 0.882194, 

...

epoch: 1, accuracy: 0.707177, precision: 0.551867, recall: 0.664224, f1: 0.526418, loss:0.492056
epoch: 1, accuracy: 0.708531, precision: 0.550992, recall: 0.660668, f1: 0.526037, loss:0.428675
epoch: 1, accuracy: 0.710562, precision: 0.551531, recall: 0.661763, f1: 0.527399, loss:0.275538
epoch: 1, accuracy: 0.707515, precision: 0.549488, recall: 0.655834, f1: 0.524060, loss:0.483815
epoch: 1, accuracy: 0.702776, precision: 0.547647, recall: 0.651136, f1: 0.520273, loss:0.331528
epoch: 1, accuracy: 0.682464, precision: 0.542739, recall: 0.640183, f1: 0.507095, loss:0.649437
epoch: 1, accuracy: 0.666554, precision: 0.539816, recall: 0.633746, f1: 0.497614, loss:0.457695
epoch: 1, accuracy: 0.655044, precision: 0.538007, recall: 0.629682, f1: 0.490999, loss:0.505918
epoch: 1, accuracy: 0.646242, precision: 0.539789, recall: 0.637794, f1: 0.488902, loss:0.361797
epoch: 1, accuracy: 0.628639, precision: 0.538732, recall: 0.636874, f1: 0.480089, loss:0.292286
epoch: 1, accuracy: 0.613744, 

epoch: 1, accuracy: 0.664861, precision: 0.552471, recall: 0.679980, f1: 0.509090, loss:0.373866
epoch: 1, accuracy: 0.631686, precision: 0.546844, recall: 0.666376, f1: 0.488735, loss:0.500931
epoch: 1, accuracy: 0.608666, precision: 0.543187, recall: 0.656106, f1: 0.474647, loss:0.529667
epoch: 1, accuracy: 0.574137, precision: 0.537394, recall: 0.637486, f1: 0.453029, loss:0.397817
epoch: 1, accuracy: 0.524374, precision: 0.531614, recall: 0.617081, f1: 0.423235, loss:0.479916
epoch: 1, accuracy: 0.501016, precision: 0.528905, recall: 0.606628, f1: 0.408924, loss:0.520374
epoch: 1, accuracy: 0.491875, precision: 0.527644, recall: 0.601699, f1: 0.403140, loss:0.727011
epoch: 1, accuracy: 0.486459, precision: 0.526903, recall: 0.598778, f1: 0.399702, loss:0.539575
epoch: 1, accuracy: 0.486459, precision: 0.526903, recall: 0.598778, f1: 0.399702, loss:0.588797
epoch: 1, accuracy: 0.514218, precision: 0.530168, recall: 0.611604, f1: 0.416861, loss:0.456915
epoch: 1, accuracy: 0.562288, 

epoch: 1, accuracy: 0.838863, precision: 0.606618, recall: 0.728806, f1: 0.629750, loss:0.474903
epoch: 1, accuracy: 0.837508, precision: 0.608975, recall: 0.738791, f1: 0.632633, loss:0.532287
epoch: 1, accuracy: 0.829045, precision: 0.607719, recall: 0.749229, f1: 0.629855, loss:0.541091
epoch: 1, accuracy: 0.819905, precision: 0.606408, recall: 0.759301, f1: 0.626382, loss:0.482095
epoch: 1, accuracy: 0.806703, precision: 0.603862, recall: 0.769326, f1: 0.619855, loss:0.575189
epoch: 1, accuracy: 0.799255, precision: 0.606571, recall: 0.788883, f1: 0.620625, loss:0.581725
epoch: 1, accuracy: 0.791131, precision: 0.606699, recall: 0.799503, f1: 0.617899, loss:0.533076
epoch: 1, accuracy: 0.783006, precision: 0.604149, recall: 0.799408, f1: 0.612032, loss:0.366500
epoch: 1, accuracy: 0.778267, precision: 0.602665, recall: 0.798996, f1: 0.608571, loss:0.646689
epoch: 1, accuracy: 0.771835, precision: 0.598258, recall: 0.789098, f1: 0.601282, loss:0.445972
epoch: 1, accuracy: 0.769127, 

epoch: 1, accuracy: 0.837847, precision: 0.624202, recall: 0.790406, f1: 0.651901, loss:0.553395
epoch: 1, accuracy: 0.837170, precision: 0.619511, recall: 0.775040, f1: 0.645921, loss:0.486555
epoch: 1, accuracy: 0.835477, precision: 0.615233, recall: 0.763412, f1: 0.640292, loss:0.494133
epoch: 1, accuracy: 0.834123, precision: 0.610495, recall: 0.749824, f1: 0.634134, loss:0.532156
epoch: 1, accuracy: 0.837508, precision: 0.608314, recall: 0.736648, f1: 0.631791, loss:0.429779
epoch: 1, accuracy: 0.846649, precision: 0.604199, recall: 0.707288, f1: 0.626923, loss:0.511997
epoch: 1, accuracy: 0.856804, precision: 0.607721, recall: 0.697764, f1: 0.630902, loss:0.638597
epoch: 1, accuracy: 0.860528, precision: 0.603457, recall: 0.678341, f1: 0.624501, loss:0.427126
epoch: 1, accuracy: 0.864929, precision: 0.601218, recall: 0.663570, f1: 0.620340, loss:0.647824
epoch: 1, accuracy: 0.867976, precision: 0.606127, recall: 0.669499, f1: 0.626044, loss:0.440749
epoch: 1, accuracy: 0.868991, 

epoch: 1, accuracy: 0.716994, precision: 0.596818, recall: 0.825959, f1: 0.574865, loss:0.398812
epoch: 1, accuracy: 0.703791, precision: 0.593691, recall: 0.820983, f1: 0.565386, loss:0.292948
epoch: 1, accuracy: 0.688219, precision: 0.589678, recall: 0.812586, f1: 0.553883, loss:0.394900
epoch: 1, accuracy: 0.663169, precision: 0.583855, recall: 0.799077, f1: 0.535880, loss:0.428954
epoch: 1, accuracy: 0.650305, precision: 0.581130, recall: 0.792140, f1: 0.526831, loss:0.376175
epoch: 1, accuracy: 0.667231, precision: 0.584197, recall: 0.799125, f1: 0.538305, loss:0.418301
epoch: 1, accuracy: 0.684834, precision: 0.588295, recall: 0.808617, f1: 0.550944, loss:0.222863
epoch: 1, accuracy: 0.703453, precision: 0.592499, recall: 0.816514, f1: 0.564155, loss:0.269119
epoch: 1, accuracy: 0.732566, precision: 0.599325, recall: 0.825784, f1: 0.584867, loss:0.421046
epoch: 1, accuracy: 0.765403, precision: 0.609735, recall: 0.839205, f1: 0.610469, loss:0.173613
epoch: 1, accuracy: 0.769127, 